In [1]:
!pip install sentence-transformers --quiet
#Define train sample.
import numpy as np
model_name="nlpaueb/legal-bert-base-uncased"

In [3]:
import pickle
import torch
with open('../input/output-ledgarlexglue-preprocesse-generate-dataset/lexglueLedgar012.pickle', 'rb') as handle:
    samples = pickle.load(handle)

train_samples=samples["train_samples"]
dev_samples=samples["dev_samples"][:1000]
print(len(train_samples))
print(len(dev_samples))

521595
1000


In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import BinaryClassificationEvaluator

word_embedding_model = models.Transformer(model_name, max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=8)
#train_loss = losses.CosineSimilarityLoss(model)
train_loss = losses.ContrastiveLoss(model=model)
# We add an evaluator, which evaluates the performance during training
evaluator = BinaryClassificationEvaluator.from_input_examples(dev_samples, name='sts-dev')
# save score 
scores=[]
epochs=[]
def callback(score, epoch, steps):
    scores.append(score)
    epochs.append(epoch)
import time
# get the start time
st = time.time()
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, callback=callback, evaluator=evaluator, evaluation_steps=10000,)
# get the end time
et = time.time()
# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
import pickle
import torch
history = {'scores':scores, 'epochs':epochs, 'elapsed_time':elapsed_time}
with open('history.pickle', 'wb') as handle:
    pickle.dump(history, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('history.pickle', 'rb') as handle:
    history = pickle.load(handle)

torch.save(model, "/kaggle/working/model_LEDGAR1.pt")

In [ ]:
print('Execution time:', history['elapsed_time'], 'seconds')
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.plot(history['scores'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Validation'], loc='upper left')
plt.show()
print(history['epochs'])